In [1]:
import sys
sys.path.append('C:\_Github\structural_engineering_toolbox')
from etabs_tools import etabs_api
import pandas as pd

<>:2: SyntaxWarning: invalid escape sequence '\_'
<>:2: SyntaxWarning: invalid escape sequence '\_'
C:\Users\zeyad.moustafa\AppData\Local\Temp\ipykernel_76692\4188593797.py:2: SyntaxWarning: invalid escape sequence '\_'
  sys.path.append('C:\_Github\structural_engineering_toolbox')


Define load combinations to be considered

In [2]:
load_combos = '(7) G+0.6Q+RS'

Create instance of ETABS api and Sap Model object

In [3]:
etabs_api = etabs_api.etabs_api()
sap_model = etabs_api.sap_model

In [4]:
piers = etabs_api.get_piers(load_cases=load_combos)
df = pd.DataFrame(piers)
df

,Pier Name,Story Name,Axis Angle,Num Area Objs,Num Line Objs,Width Bot,Thickness Bot,Width Top,Thickness Top,Mat Prop,CG Bot X,CG Bot Y,CG Bot Z,CG Top X,CG Top Y,CG Top Z,Story Height,fc
0,SOUTH CORE 1,LEVEL 9,90.0,2,0,3200.0,350.0,3200.0,481.25,C50,13200.0,1300.0,31200.0,14918.181818,1736.363636,34200.0,3000000.0,50.0
1,SOUTH CORE 1,LEVEL 8,90.0,2,0,3200.0,350.0,3200.0,481.25,C50,13200.0,1300.0,28200.0,14918.181818,1736.363636,31200.0,3000000.0,50.0
2,SOUTH CORE 1,LEVEL 7,90.0,2,0,3200.0,350.0,3200.0,481.25,C50,13200.0,1300.0,25200.0,14918.181818,1736.363636,28200.0,3000000.0,50.0
3,SOUTH CORE 1,LEVEL 6,90.0,2,0,3200.0,350.0,3200.0,481.25,C50,13200.0,1300.0,21000.0,14918.181818,1736.363636,25200.0,4200000.0,50.0
4,SOUTH CORE 1,LEVEL 5,90.0,2,0,3200.0,350.0,3200.0,481.25,C50,13200.0,1300.0,16800.0,14918.181818,1736.363636,21000.0,4200000.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,P23,LEVEL 5,90.0,1,0,3200.0,350.0,3200.0,350.00,C50,20100.0,1300.0,16800.0,20100.000000,1300.000000,21000.0,4200000.0,50.0
180,P23,LEVEL 4,90.0,1,0,3200.0,350.0,3200.0,350.00,C50,20100.0,1300.0,12600.0,20100.000000,1300.000000,16800.0,4200000.0,50.0
181,P23,LEVEL 3,90.0,1,0,3200.0,350.0,3200.0,350.00,C50,20100.0,1300.0,8400.0,20100.000000,1300.000000,12600.0,4200000.0,50.0
182,P23,LEVEL 2,90.0,1,0,3200.0,350.0,3200.0,350.00,C50,20100.0,1300.0,4200.0,20100.000000,1300.000000,8400.0,4200000.0,50.0
